## Setting up a Hydration Free Energy Calculuation
This notebook will guide you through how to use FESetup in order to run an alchemical free energy calculation.    


The notebook forms part of the CCPBio-Sim workshop **Tackling Alchemistry with FESetup and Sire SOMD** run on the 10th of April 2018 at the University of Bristol.

*Author: Anotnia Mey   
Email: antonia.mey@ed.ac.uk*

**Reading time of the document: xx mins**

### Setting up the notebook with all necessary imports

In [ ]:
%pylab inline
import nglview as nv

### Setting the stage
We want to compute the relative free energy of hydration between ethane and methanol. That is the free energy difference between hydrating an ethane molecule in water and a methanol molecule in water. It assumes that you had a look at the slides of lecture 1 or attended lecture1 (maybe youtube link here).

Below you can see a thermodynamic cycle for the relative hydration free energy of ethane and methanol:
![therm_cycle](images/Therm_cycle.png)

We use FESetup to authomatically generate the perturbation information needed for running the alchemical free energy calcualtions.

### Visualise molecule

Have a look in the `FESetup_01` directory. You will find a file called `morph.in` and a directory called `solute`. `morph.in` is the main input file and we will look at it more closely in a second. `solute` contains two further directories: `ethane` and `methanol` each containing a pdb structure file called `solute.pdb` containing the 3D structure of the two molecules we want to compute the relative hydration free energy difference.   

Let's use nglview to look at the ethane molecule:

In [ ]:
view = nv.show_file('FESetup_01/solutes/ethane/solute.pdb')
view

**1. Task: load methanol into nglview as well**

In [ ]:
#Insert code for methanol here



## FESetup
Now, rather than just looking at the structure of these two molecules we want to set them up with FESetup such that they will be ready for running an alchemical free energy simulation. You can open a terminal and run FESetup from the command line by simply typing:   
    ```FESetup```

Or you can run things within the notebook. In order to access command line arguments in a notebook you can use the exclamation mark as an escape and the magic function %%capture to capture all output. 

In [ ]:
%%capture info
!FESetup 

The reason why it is useful to capture output is because sometimes quite a lot of output is generated. This is also the case for FESetup. If you run FESetup without providing it with an input file, it will list all possible config file options you might want to use. We can access the captured output by printing it in the following way:

In [ ]:
print (info.stdout)

FESetup is a command line tool and you can pass it an input file defining what you would like to set up. Let's change to the directory `FESetup_01` in order to run our fist setup. 

In [ ]:
cd FESetup_01/

In [ ]:
ls

We will be working with ethane and methanol. Let's have a closer look at the inputfile `morph.in` provided.

In [ ]:
!head -n 7 morph.in

The first section consists of global directives. Important here are the two directives `AFE.tye=Sire` and `AFE.separate_vdw_elec = false`. The first one means that Sire compatible output is generated and the second one means that the simulation will be carried out using a single step perturbation, perturbing charges and van der Waals interactions at the same time. Later on we will also look at how to generate output for different simulation software.

In [ ]:
!tail -n 19 morph.in

For FESetup to understand what files to work on, a strict directory structure is used. `basdir` gives information where it can find all molecules files. Here the basedir is called `solutes`. In `solutes` you have the molecules ethane and methanol as indicated by the corresponding directory and the actual filename which should be the same for all solutes and is in this case `file.name = solute.pdb`. `molecules` defines the molecules that should actually be setup and just lists the directories where we have put the `solutes.pdb` files.

Any directive that starts with `box.type` indicates that a solvated version of the molecule should be generated. FESetup also allows to run a minimisation straight away by using the different `min` directives. In the above example a minimisation for 100 steps using a restraint on non-solvent molecules of 10 will be run. 

To execute FESetup you either type into your terminal `FESetup morph.in` or execute the next cell.

In [ ]:
!FESetup morph.in

Let's have a look at the directory again. Some additional files have been generated!

In [ ]:
ls

Directories with an underscore `_` are output directories that contain the relevant information for the simulation input. The most useful file is the generated logfile `eth-meth.log`. It contains all commands that were used in order to actually generate the output.  It also contains information whether a setup has failed or not. If you are familiar with AMBER setup tools you will find a lot of familiar looking commands. If you want you can use `nano` on the commandline in order to take a closer look. 

### Solvated molecules
The generated directory `_ligands` contains the original pdb files in a box of water, found in `_ligands/ethane` and `_ligands/methanol`. Let's have a more detailed look at one of the directories. 

In [ ]:
ls _ligands/methanol/

If you wanted to run a simple MD simulation of methanol in water you can use the `solvated.rst7` (coordinate file) and `solvated.parm7` (topology file) files. The coordinate file contains the energy minimised coordinates as generated by the minimisation protocol in the `morph.in` file. But since we are interested in running an alchemical free energy perturbation from ethane to methanol `_perturbations` is actually the directory we are interested in. 

**Task visualise the solvated molecule `_ligands/methanol/solvated.pdb` using nglview below**

### Perturbation files
Let's take a closer look at `_perturbations`. It has a subdirectory called `sire`, because we chose Sire to be the alchemical free energy simulation engine and that again has a subdirectory called `ethane~methanol` indicating the perturbation we want to simulate. 

In [ ]:
ls _perturbations/sire/ethane~methanol/

The files of interest are the `MORPH.onestep.pert` file, `vacuum.parm7` (topology), `vacuum.rst7` (coordinates) and in the directory `solvated`, the solution equivalent `solvated.parm7` and `solvated.rst7` files. The solution and vacuum files will be used to simulate the vacuum and solution leg of the thermodynamic cycle. Let's take a closer look at `MORPH.onestep.pert`

In [ ]:
!head -n11 _perturbations/sire/ethane~methanol/MORPH.onestep.pert

It defines the initial and final atom types and their initial and final Lennard Jones parameters as well as partial charges. In the perturbation between ethane and methanol the first C atom (C1), will stay a C atom but the partial charges on the molecule will change because of the changed environment from initially having a methyl group attached to it to an OH group attached after the transformation. The second C atom will change to being an oxygen atom after the perturbation. So lets have a look at that atom in the `.pert` file. 

In [ ]:
!sed -n '12,20p' _perturbations/sire/ethane~methanol/MORPH.onestep.pert

You can now see how the initial atom type is `c3`, and the final atom type is `oh`, meaning that we are going from an sp3 hybridised carbon atom to an oxygen atom that is bonded to a hydrogen. The Lennard Jones terms change a little and the partial charges do too. The alchemical free energy software can then create a linear perturbation between the initial and final values to slowly morph from one to the other.  

### Adding more perturbations
Now we understand the output generated by FESetup lets see how we can change the input to get a different output. 

**Task: Can you modify the `morph.in` file in such a way that we not only generate a ethane to methanol, but also a methanol to ethane perturbation file**

Using nano open morph.in and identify what you need to change in order to not only generate ethane > methanol, but also methanol > ethane perturbation. Hint: Lists of perturbations are separated by a comma. 

In [ ]:
## Now run the modified file through FESetup add the requried code below


We shoud now have an additional perturbation in the `_perturbations/sire` directory. 

In [ ]:
ls _perturbations/sire

**Task: visulatise the `vacuum.pdb` file using nglview of the new methanol~ethane perturbation**

In [ ]:
## add code to load vacuum.pdb into nglview



Questions:
- Why does the molecule look so weird, have we done something wrong?
- Can you identify the atom types that are connected to oxygen atom? Think of what was covered in the lecture. 

### Equilibration in FESetup
FESetup can also be used to straightaway equilibrate the solvation box with the solute in it. It may be desirable to run an equilibration protocol with FESetup, this can be done by adding equilibration information to the input file. For this purpose a second FESetup directory FESetup_02 has been prepared. Change to that directory and have a look at the `equilibrate.in` input file. 

In [ ]:
cd ../FESetup_02/

In [ ]:
!sed -n '10,50p' equilibrate.in

The file now contains a set of directives that will run a NVT simulation to heat the system to 300 K followed by an NPT simulation to equilibrate the density and then a protocol that will slowly release the restraint on the solute molecule in the NPT ensemble. Note however, how we have not specified any kind of morphing. It is often useful to not have the morph and equilibration happen in one go, because often you want to run the equilibrations individually in parallel on a cluster and then when completed actually run the morphing. 

In [ ]:
# Insert code to run the equilibration. The calculation should run for about 2-3 minutes. 



**Do you get an error?** Can you trouble shoot what it might be?

With the equilibration successfully run, we can now run the morphing. You should notice that there is no `_perturbations` directory that was created by FESetup. 

In [ ]:
ls

This step will be very quick. For this purpose we have a `morph.in` file again. Have a look at it. It is much shorter and mostly defines the type of simulation engine. In order to make sure that we also want to morph the solution phase and not just the vacuum phase is indicated by specifying the `box.type = rectangular` and `box.length = 12.0` directives. 

In [ ]:
# Insert the code to then run the morphing



Make sure that the perturbations were actually successfully generated. 

In [ ]:
ls _perturbations/sire

### Running simulation in vacuum
We will now run a set of ethane to methanol simulations in vacuum using the `ethane~methanol` perturabtion. Let's change to the simulation directory. 

In [29]:
cd ../simulation/

/home/jovyan/CCP-BioSim-Workshop/Exercises/01_FESetup_ethane_methanol/simulation


There are some files prepared for you in the `simulation` directory.  Let's have a closer look at these:

In [31]:
ls 

input/  output/


So what do we need for input to run an alchemical free energy simulation? Basically a bunch of files we have generated with FESetup and a config file. To be precise:

- a config file
- a coordinate file
- a topology file
- a morph file

Currently we only have the config file in the input directory:

In [33]:
ls input/

MORPH.pert  sim.cfg  vacuum.parm7  vacuum.pdb  vacuum.rst7


Let's copy the ethane~methanol vacuum files we have generated with FESetup into the input directory:

In [37]:
!cp ../FESetup_01/_perturbations/sire/ethane~methanol/vacuum.* input/
!cp ../FESetup_01/_perturbations/sire/ethane~methanol/MORPH*.pert input/MORPH.pert

We now have all the requried input files:

In [38]:
ls input/

MORPH.pert  sim.cfg  vacuum.parm7  vacuum.pdb  vacuum.rst7


The `parm7` file is the topology file and the `rst7` file is the coordinate file in standard AMBER format. The `pdb` file we don't really need, but it is nice to have a quick pdb structure available for visualisation. The `MORPH.pert` is the important file for the perturbation as discussed above. Let's just look briefly at the config file though:

In [40]:
!head -n20 input/sim.cfg

nmoves = 10000 
ncycles = 5 
buffered coordinates frequency = 500 
save coordinates = True 
timestep = 2*femtosecond 
constraint = hbonds-notperturbed
cutoff type = nocutoff 
barostat = False 
andersen = True 
energy frequency = 100 
precision = mixed 
minimise = True 
equilibrate = False 
center solute = True
minimal coordinate saving = True
lambda array = 0.00, 0.30, 0.60, 1.00


We will be running a `nmoves`\*`ncycles`\*`timestep` simultation, i.e. in this case 100 ps. Because we are in vacuum we don't use a periodic box and have no barostat, but set an Anderson thermostat. The default simulation temperature is 298 K. However you could set the temperature yourself. We will minimise the molecule again before simulation as indicated by `minimise=True`. The most important information again is the `lambda array`. In this case we will simulate at 4 different lambda values with the MORPH file being used to create a linear interpolation of intermediates. If you want know the defaults of all config file options and what all available config file options are for running the alchemical free energy simulation then you can simply run:

In [42]:
!somd-freenrg --help-config

Starting somd-freenrg: number of threads equals 8

andersen frequency = 10.0
Collision frequency in units of (1/ps)

barostat = True
Whether or not to use a barostat (needed for NPT simulation).

barostat frequency = 25
Number of steps before attempting box changes if using the barostat.

buffered coordinates frequency = 1
The number of time steps between saving of coordinates during a cycle of MD. 0 disables buffering.

center of mass frequency = 10
Frequency of which the system center of mass motion is removed.

center solute = False
Whether or not to centre the centre of geometry of the solute in the box.

combining rules = arithmetic
Combining rules to use for the non-bonded interactions.

constraint = hbonds
The constraint model to use during dynamics.

coulomb power = 0
Value of the Coulombic soft-core parameter.

crdfile = SYSTEM.crd
File name of the coordinate file containing the coordinates of the system to be simulated.

cutoff distance = 10 angstrom
The cutoff distance to us

Next, we can look at the ouptput directory which has been set up already for convenience.

In [46]:
ls output

lambda-0.00/  lambda-0.30/  lambda-0.60/  lambda-1.00/  run_simulation.sh


Four directories for the output of all the different lambda simlations has been setup already and are currently empty. A convenient script to serially execute simulations is also provided with `run_simulation.sh`. 

In [47]:
!head output/run_simulation.sh

#!/bin/bash

for l in 0.00 0.30 0.60 1.00
do
cd lambda-$l
somd-freenrg -C ../../input/sim.cfg -c ../../input/vacuum.rst7 -t ../../input/vacuum.parm7 -m ../../input/MORPH.pert -p CPU -l $l
cd ../
done 


In `run_simulation.sh` we are looping over the four lambda directories and run an alchemical free energy calculation with the input. The executable is called `somd=freenrg` and forms part of Sire. The flag `-C` indicates the config file that is being passed, the flag `-c` the coordinate file `-t` the topology file and `-m` the morph file. The platform used for the simulation is indicated by `-p`. Normally it is advised to run simulations on GPUs and either use `CUDA` or `OpenCL` as a platform. These are currently not available on the cloud we are running on. For this reason we are using the VERY slow `CPU` platform. If you go back and want to actually run some meaningful simulations make sure these are done on GPUs such as the Nvidia GTX series. The last flag `-l` passes the current lambda value of the simulation.  

We can now go ahead and run our short set of simulations. They should run for around 15 minutes. 

In [48]:
cd output

/home/jovyan/CCP-BioSim-Workshop/Exercises/01_FESetup_ethane_methanol/simulation/output


In [49]:
!bash run_simulation.sh

Starting somd-freenrg: number of threads equals 8

Loading configuration information from file ../../input/sim.cfg

Running a somd-freenrg calculation using files ../../input/vacuum.parm7, ../../input/vacuum.rst7 and ../../input/MORPH.pert.
Using parameters:
andersen == True
barostat == False
buffered coordinates frequency == 500
center solute == True
constraint == hbonds-notperturbed
crdfile == ../../input/vacuum.rst7
cutoff type == nocutoff
energy frequency == 100
equilibrate == False
lambda array == (0.0, 0.3, 0.6, 1.0)
lambda_val == 0.0
minimal coordinate saving == True
minimise == True
morphfile == ../../input/MORPH.pert
ncycles == 5
nmoves == 10000
platform == CPU
precision == mixed
save coordinates == True
timestep == 0.002 ps
topfile == ../../input/vacuum.parm7
### Running Single Topology Molecular Dynamics Free Energy on jupyter-ad44563a-2d7a4c-2d42ac-2d8dd0-2dda77190744a9 ###
###================Setting up calculation=====================###
New run. Loading input and creating

**Well done! You have now started running a short simulation with a free energy perturbation in vacuum, let's go and have a tea/coffe while we wait for this to finish!**

## What has been generated?
Each lamabda directory now contains relevant output. You can use NGLview to visualise an intermediate lambda trajectory for example. 

In [ ]:
import MDAnalysis as mda
coor_file = '../input/vacuum.pdb'
trajectory_file = 'lambda-0.30/traj.traj000000001.dcd'
u = mda.Universe(coor_file, trajectory_file)
ethane = u.select_atoms('all')
view = nv.show_mdanalysis(ethane)